# Car Price Analysis

This Jupyter notebook performs exploratory data analysis (EDA), preprocessing, and basic price prediction modeling on a **Car Price Analysis** dataset.  
It also includes market-trend analysis (average price over years, price by make/vehicle type) and a simple model comparison.

**Files included in this repo**
- `data/car_price_sample.csv` — synthetic sample dataset (300 rows) for demonstration.
- `scripts/train_model.py` — standalone script that trains and saves a model.


In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
plt.rcParams['figure.figsize'] = (8,5)

In [ ]:
# Load dataset
df = pd.read_csv('data/car_price_sample.csv')
df.head()

## Quick summary statistics

In [ ]:
df.describe(include='all')

## Missing values (should be none in the sample)

In [ ]:
df.isna().sum()

## Simple visualizations: price distribution and price by year

In [ ]:
plt.hist(df['price'], bins=30)
plt.title('Price distribution')
plt.xlabel('Price (USD)')
plt.show()

df.groupby('year')['price'].mean().plot()
plt.title('Average price by year')
plt.xlabel('Year')
plt.ylabel('Average Price')
plt.show()

## Preprocessing and model pipeline

In [ ]:
# Features and target
X = df.drop(columns=['price'])
y = df['price']

# Identify categorical and numeric features
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()

# Build preprocessing
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Model pipelines
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', RandomForestRegressor(n_estimators=100, random_state=42))])

lr_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_pipeline.fit(X_train, y_train)
lr_pipeline.fit(X_train, y_train)

for name, pipe in [('RandomForest', rf_pipeline), ('LinearRegression', lr_pipeline)]:
    preds = pipe.predict(X_test)
    print(name)
    print(' MAE:', mean_absolute_error(y_test, preds))
    print(' R2:', r2_score(y_test, preds))
    print()

## Market trend analysis examples

In [ ]:
# Avg price by make
avg_by_make = df.groupby('make')['price'].mean().sort_values(ascending=False)
print(avg_by_make)

# Avg price by vehicle_style
print('\nAverage price by vehicle style:')
print(df.groupby('vehicle_style')['price'].mean())

## Save a simple model (RandomForest) for later use

In [ ]:
import joblib
joblib.dump(rf_pipeline, 'car_price_rf_pipeline.joblib')
print('Saved model to car_price_rf_pipeline.joblib')

## Next steps / Improvements

- Use a larger real-world dataset (Kaggle, UCI) to improve coverage and realism.
- Add feature engineering (age, mileage if available, brand-brand interactions).
- Hyperparameter tuning and cross-validation.
- Interpretability: SHAP values or partial dependence plots.
- Deployment: build a small Flask/FastAPI service to serve predictions.